In [1]:
import pandas as pd
import os
import geopandas as gpd
from pymongo import MongoClient
import requests
import json 
from dotenv import load_dotenv
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [2]:
client = MongoClient("mongodb://localhost/ironhack")
db = client.get_database()

In [5]:
list(db.list_collections())

[{'name': 'companies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('f0df70fa-ecbb-4210-b11a-d2f3d3075dc8')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [6]:
db.companies.distinct("category_code") 

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

I make a dataframe with all of the web companies, I use London because is one of the European cities with the most tech companies.